In [42]:
import pandas as pd
import folium
import requests

launch_url = "https://api.spacexdata.com/v4/launches"
pad_url = "https://api.spacexdata.com/v4/launchpads"

launch_res = requests.get(launch_url)
pad_res = requests.get(pad_url)

if launch_res.status_code == 200 and pad_res.status_code == 200:
    launches = pd.json_normalize(launch_res.json())
    pads = pd.json_normalize(pad_res.json())
    print("Data loaded successfully!")
else:
    print("Failed to fetch API data")
    exit()

pad_coords = pads[['id', 'name', 'locality', 'latitude', 'longitude']].copy()
pad_coords.dropna(subset=['latitude', 'longitude'], inplace=True)

pad_coords.rename(columns={'name': 'pad_name'}, inplace=True)
launches.rename(columns={'name': 'mission_name'}, inplace=True)

launches['launchpad'] = launches['launchpad'].astype(str)
merged_df = pd.merge(launches, pad_coords, left_on='launchpad', right_on='id')

m = folium.Map(location=[28.5, -80.6], zoom_start=3, tiles='CartoDB positron')

for _, row in merged_df.iterrows():
    location = [row['latitude'], row['longitude']]
    popup_text = f"""
    <b>Mission:</b> {row['mission_name']}<br>
    <b>Launchpad:</b> {row['pad_name']}<br>
    <b>Locality:</b> {row['locality']}<br>
    <b>Date:</b> {row['date_utc'][:10]}<br>
    <b>Success:</b> {row['success']}
    """
    color = "green" if row['success'] else "red"

    folium.Marker(
        location=location,
        popup=popup_text,
        icon=folium.Icon(color=color)
    ).add_to(m)

    folium.Circle(
        location=location,
        radius=10000,
        color=color,
        fill=True,
        fill_opacity=0.1
    ).add_to(m)

spacex_hq = [33.9207, -118.3280]
for _, row in merged_df.iterrows():
    folium.PolyLine(
        locations=[[row['latitude'], row['longitude']], spacex_hq],
        color="blue",
        weight=1,
        opacity=0.4
    ).add_to(m)

m  


Data loaded successfully!
